## 📊 Modelado Final: XGBoost Optimizado para Predicción de Abandono Estudiantil

Este notebook implementa el modelo final de XGBoost con hiperparámetros optimizados, evaluación profesional (Classification Report, Matriz de Confusión, Curva ROC y AUC) e interpretación de features.

### Objetivos:
- Entrenar el modelo final con los mejores hiperparámetros.
- Evaluarlo con métricas robustas para validar el rendimiento (NFR-01).
- Generar visualizaciones y Exhibits para el informe.
- Interpretar qué features son clave para las predicciones (FR-03).


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# Configuración para gráficos
sns.set(style='whitegrid')
plt.rcParams['figure.figsize'] = (10, 6)


### Paso 1: Carga y Preparación de Datos

Cargamos los datos procesados, binarizamos la variable objetivo para enfocarnos en abandono y dividimos en train/test.


In [ ]:
# Cargar datos procesados
df = pd.read_parquet('../data/processed/preprocessed_data.parquet')

# Definir features (X) y target (y)
X = df.drop('Target', axis=1)
y = df['Target']

# Binarizar y: 1 si 'Dropout', 0 si no (para enfocarnos en abandono)
y = y.apply(lambda x: 1 if x == 'Dropout' else 0)

# Label encoding para convertir a numérico
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

# Dividir en train y test con estratificación
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print(f'Datos cargados: {df.shape}')
print(f'Train: {X_train.shape}, Test: {X_test.shape}')
print(f'Balance de clases en train: {pd.Series(y_train).value_counts(normalize=True)}')


### Paso 2: Entrenamiento del Modelo Final

Usamos los mejores hiperparámetros encontrados en la optimización previa.


In [ ]:
# Mejores hiperparámetros encontrados
best_params = {'n_estimators': 200, 'max_depth': 5, 'learning_rate': 0.1}

# Instanciar y entrenar el modelo final (binario para abandono)
model = XGBClassifier(
    objective='binary:logistic',
    eval_metric='logloss',
    use_label_encoder=False,
    random_state=42,
    **best_params
)

model.fit(X_train, y_train)
print('Modelo final entrenado exitosamente.')


### Paso 3: Evaluación Profesional del Modelo

Calculamos métricas clave: Classification Report, Matriz de Confusión, Curva ROC y AUC.


In [ ]:
# Predicciones en el conjunto de test
y_pred = model.predict(X_test)
y_pred_proba = model.predict_proba(X_test)[:, 1]  # Probabilidades para ROC

# 1. Classification Report
print('--- Classification Report ---')
report = classification_report(y_test, y_pred, target_names=['No Dropout', 'Dropout'])
print(report)

# 2. Matriz de Confusión
print('\n--- Matriz de Confusión ---')
cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=['No Dropout', 'Dropout'], yticklabels=['No Dropout', 'Dropout'])
plt.xlabel('Predicción')
plt.ylabel('Realidad')
plt.title('Matriz de Confusión - Modelo Final')
plt.savefig('../exhibits/matriz_confusion.png')  # Guarda el Exhibit
plt.show()

# 3. Curva ROC y AUC
print('\n--- Curva ROC y AUC ---')
fpr, tpr, thresholds = roc_curve(y_test, y_pred_proba)
roc_auc = auc(fpr, tpr)
print(f'Área Bajo la Curva (AUC): {roc_auc:.4f}')

plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'Curva ROC (AUC = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('Tasa de Falsos Positivos')
plt.ylabel('Tasa de Verdaderos Positivos')
plt.title('Curva ROC - Modelo Final')
plt.legend(loc='lower right')
plt.savefig('../exhibits/curva_roc.png')  # Guarda el Exhibit
plt.show()


### Paso 4: Interpretación del Modelo (Feature Importance)

Extraemos y visualizamos las características más importantes del modelo final.


In [ ]:
# Feature Importance
feature_importances = pd.DataFrame({
    'Feature': X_train.columns,
    'Importance': model.feature_importances_
}).sort_values('Importance', ascending=False)

# Top 15 features
top_features = feature_importances.head(15)

print('--- Top 15 Características Más Importantes ---')
print(top_features)

# Visualización
plt.figure(figsize=(10, 8))
sns.barplot(x='Importance', y='Feature', data=top_features)
plt.title('Top 15 Características Más Importantes - Modelo Final')
plt.xlabel('Importancia')
plt.ylabel('Característica')
plt.savefig('../exhibits/feature_importance.png')  # Guarda el Exhibit
plt.show()

# Interpretación breve
print('\n--- Interpretación ---')
print(f'La característica más importante es "{top_features.iloc[0]["Feature"]}" con importancia {top_features.iloc[0]["Importance"]:.4f}.')
print('Esto indica que factores como el rendimiento académico y financieros son clave para predecir abandonos.')


### Conclusiones Finales

- **Rendimiento del Modelo:** Revisa el AUC y F1-Score. Un AUC >0.8 indica un modelo sólido para distinguir entre graduados y desertores.
- **Exhibits Generados:** Los gráficos se guardaron en `../exhibits/` para tu informe.
- **Próximos Pasos:** Usa estos resultados para validar NFR-01 y explicar el modelo en tu presentación.

¡Ejecuta las celdas en orden y listo!
